In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import numpy.typing as npt
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import skimage as ski

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
        

datapath_train = "./mnist_data/mnist_train.csv"
datapath_test = "./mnist_data/mnist_test.csv"

datatype = np.float64

train_data = np.loadtxt(datapath_train, dtype=datatype, delimiter=",", skiprows=1)
test_data = np.loadtxt(datapath_test, dtype=datatype, delimiter=",", skiprows=1)
print(f"Shape of training data: {train_data.shape}")
print(f"Shape of test_data: {test_data.shape}")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Shape of training data: (60000, 785)
Shape of test_data: (10000, 785)


In [3]:
train_labels: npt.NDArray = train_data[:, 0]
print(f"Shape of train labels: {train_labels.shape}")
train_vals = train_data[:, 1:] / 255
print(f"Shape of train values: {train_vals.shape}")


test_labels: npt.NDArray = test_data[:, 0]
print(f"Shape of test labels: {test_labels.shape}")
test_vals = test_data[:, 1:] / 255
print(f"Shape of test values: {test_vals.shape}")

Shape of train labels: (60000,)
Shape of train values: (60000, 784)
Shape of test labels: (10000,)
Shape of test values: (10000, 784)


In [4]:
def sigmoid(x: npt.NDArray) -> npt.NDArray:
    return 1 / (1 + np.exp(-x))

def mse_loss(predictions: npt.NDArray, labels: npt.NDArray):
    truth_table = np.identity(10, dtype=datatype)
    y_true = truth_table[labels.astype(dtype=np.uint8)]
    n = len(y_true)
    sum_each_column = np.sum(((predictions - y_true) ** 2) / n, axis=1)
    return sum_each_column

def ReLU(x: npt.NDArray) -> npt.NDArray:
    """ReLU(x) = max(0, x)"""
    return x * (x > 0)

def percentages(x: npt.NDArray) -> npt.NDArray:
    return np.exp(x) / np.sum(np.exp(x))

def random_weights(dimensions: npt.ArrayLike, low: float = -1, high: float = 1):
    range = high - low
    return np.random.rand(*dimensions) * range + low

def print_percentages(percentages_array: npt.NDArray) -> None:
    for i, p in enumerate(percentages_array):
        print(f"{i} = {round(p * 100, 3)}%")

In [5]:
num_mid_layer = 1000
hidden_layer_size = 100

# 784 -> `hidden_layer_size` -> 10

hidden_layer_weights = random_weights((hidden_layer_size, 784)) # Random values in the range [-1, 1)
hidden_layer_bias = random_weights([hidden_layer_size])

output_layer_weights = random_weights((10, hidden_layer_size))
output_layer_bias = random_weights([10])

hidden_layer_activations = ReLU((hidden_layer_weights @ train_vals[0].T) + hidden_layer_bias)
output_layer_activations = sigmoid((output_layer_weights @ hidden_layer_activations.T) + output_layer_bias)

print(f"Truth: {train_labels[0]}")
print_percentages(percentages(output_layer_activations))


Truth: 5.0
0 = 14.597%
1 = 14.597%
2 = 5.4%
3 = 14.597%
4 = 5.403%
5 = 5.37%
6 = 14.597%
7 = 5.47%
8 = 14.597%
9 = 5.37%


In [6]:
np.sum(mse_loss(predictions, train_labels))

NameError: name 'predictions' is not defined

In [ ]:
MAX_ITERATIONS = 1000
MIN_LOSS = np.sum(loss(predictions, train_labels))
for i in range(MAX_ITERATIONS):
    new_model = (np.random.rand(784,10) - 0.5) * 2 # Random values in the range [-1, 1)    
    new_predictions = sigmoid(train_vals.T @ new_model)
    new_loss = np.sum(loss(new_predictions, train_labels))
    if new_loss < MIN_LOSS:
        MIN_LOSS = new_loss
        new_model = model
        print(f"New model found! New loss: {MIN_LOSS}")
    else:
        print(f"Iteration {i} loss = {new_loss}")

Iteration 0 loss = 4.581399954494977
Iteration 1 loss = 4.613280041924473
New model found! New loss: 3.9159558033012525
Iteration 3 loss = 4.1407764306675405
Iteration 4 loss = 4.544472578421632
Iteration 5 loss = 5.263302532772334
New model found! New loss: 3.8935145021478643
Iteration 7 loss = 4.847840031700671
Iteration 8 loss = 4.145850498418171
Iteration 9 loss = 4.184187073433147
Iteration 10 loss = 4.084961245252244
New model found! New loss: 3.2380094638650325
Iteration 12 loss = 4.87713218892259
Iteration 13 loss = 4.599684967210228
Iteration 14 loss = 3.5842888841318854
Iteration 15 loss = 4.409398341874957
Iteration 16 loss = 3.5176616441756465
Iteration 17 loss = 3.4149532938473492
Iteration 18 loss = 5.106767460656234
Iteration 19 loss = 5.115765641041121
Iteration 20 loss = 4.723975205763852
Iteration 21 loss = 3.718606878629781
Iteration 22 loss = 3.6759505342788246
Iteration 23 loss = 4.0186701540349015
Iteration 24 loss = 4.274554085918562
Iteration 25 loss = 4.3646221

In [ ]:
print(test_vals.T[0].shape)
print(model.shape)
print(sigmoid(test_vals.T[0] @ model))
print(test_labels[0])

(784,)
(784, 10)
[0.7233194  0.98752347 0.06930624 0.03754042 0.9905272  0.01467198
 0.27637816 0.96771305 0.13265511 0.03477478]
7.0


In [ ]:
class Network:

    def __init__(self):
        self.input_layer_num = 784
        self.output_layer_num = 10
        self.hidden_layer_nums = [10, 10]